## Examples

In [2]:
from thecodecrate_pipeline import (
    Pipeline,
)


pipeline = (
    (Pipeline[int, str]())
    .pipe(lambda x: x + 1)
    .pipe(lambda x: x + 1)
    .pipe(lambda x: f"result is {x}")
)

# 'result is 3'
await pipeline.process(1)

'result is 3'

In [3]:
from thecodecrate_pipeline import (
    StageInterface,
)


class TimesTwoStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload * 2


class AddOneStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload + 1


my_pipeline = (Pipeline[int]()).pipe(TimesTwoStage()).pipe(AddOneStage())

# returns 21
await my_pipeline.process(10)

21

In [4]:
class TimesThreeStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload * 3


class AddFiveStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload + 5


pipeline = (
    (Pipeline[int]())
    .pipe(TimesThreeStage())
    .pipe(my_pipeline)
    .pipe(AddFiveStage())
)

# returns 36
# 5 * 3 = 15 * 2 + 1 = 31 + 5 = 36
await pipeline.process(5)

36

In [5]:
class MyPipeline(Pipeline[int]):
    stages = [
        TimesThreeStage(),
        my_pipeline,
        AddFiveStage,
    ]

await MyPipeline().process(5)

36

In [6]:
###
## Stages with Custom Arguments
##
## On this example, the stages have the current index
## of the processing queue
###
from abc import abstractmethod
from typing import Awaitable, Callable, Concatenate, cast

from thecodecrate_pipeline import (
    Processor,
    Stage,
    T_in,
    T_out
)


class IndexedStage(Stage[T_in, T_out]):
    @abstractmethod
    async def __call__(
        self,
        payload: T_in,
        tag: int,
    ) -> T_out:
        pass


IndexedPipelineCallable = (
    IndexedStage[T_in, T_out]
    | Callable[Concatenate[T_in, ...], Awaitable[T_out]]
    | Callable[Concatenate[T_in, ...], T_out]
)


class IndexedProcessor(Processor[T_in, T_out]):
    async def process(
        self,
        payload: T_in,
        stages: list[IndexedPipelineCallable],
    ) -> T_out:
        index = 0

        for stage in stages:
            payload = await self._call_stage(
                payload=payload,
                stage=stage,
                index=index,
            )

            index += 1

        return cast(T_out, payload)


class IndexedPipeline(Pipeline[T_in, T_out]):
    processor_class = IndexedProcessor


###
## Usage
###
class MyIndexedStage(IndexedStage[str]):
    # type-hinting: change `index` type to see an error
    async def __call__(self, payload: str, index: int) -> str:
        return f"{payload}: {index}"


indexed_pipeline = (
    (IndexedPipeline[str]()).pipe(MyIndexedStage()).pipe(MyIndexedStage())
)

assert await indexed_pipeline.process("test") == "test: 0: 1"

# returns "hello: 0: 1"
await indexed_pipeline.process("hello")

'hello: 0: 1'

In [7]:
from thecodecrate_pipeline import StatefulChainedPipeline


pipeline = (
    StatefulChainedPipeline[int]()
    .pipe(TimesThreeStage())
    .pipe(AddFiveStage())
)

# returns 20
await pipeline.process(5)

20

In [8]:
from thecodecrate_pipeline import InterruptiblePipeline

def continues_when_less_than_ten(payload: int) -> bool:
    return payload < 10

pipeline = (
    InterruptiblePipeline[int](continues_when_less_than_ten)
    .pipe(lambda payload: payload + 1)
    .pipe(lambda payload: payload * 2)
    .pipe(lambda payload: payload * 3)
)

# returns 12
await pipeline.process(5)

12

In [9]:
from thecodecrate_pipeline import Command


class MyCommand(Command[T_in, T_out]):
    async def execute(self) -> T_out:
        for stage in self.stages:
            print("Processing stage", stage)
            self.payload = await self._call_stage(
                payload=self.payload,
                stage=stage,
                *self.extra_args,
                **self.extra_kwds,
            )

        return cast(T_out, self.payload)


class MyPipeline(Pipeline[T_in, T_out]):
    command_class = MyCommand

pipeline = (
    MyPipeline[int, int]()
    .pipe(lambda payload: payload + 1)
    .pipe(lambda payload: payload * 2)
    .pipe(lambda payload: payload * 3)
)

# returns 36
await pipeline.process(5)

Processing stage <function <lambda> at 0x78902ff9e660>
Processing stage <function <lambda> at 0x78902ff9e020>
Processing stage <function <lambda> at 0x78902ff9f4c0>


36